Goal here:

- Show how to modify a method within the pygeodyn class.
- increase the number of drag outputs

Figure out:
- extend the prediction window
- extend the arc length??


Put the following plots together:
- In-track component orbit determination + prediction
- drag coefficient adjustment


In [1]:
import copy
import sys
import os.path
import os
import numpy as np

In [2]:
### Identify which arcs you want to run:


sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')

#------ A dictionary containing the run parameters ------  
run_params = {} 
run_params['arc']              =   ['2018.319']    #['2018.317','2018.318','2018.319', '2018.320' ]
run_params['satellite']        =  'icesat2'  
run_params['SpecialRun_name']  =  '_TrajAnalysis_IncreaseTimeDepDrag'  
run_params['verbose']          =  False
run_params['action']           =  'run'


In [3]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

#### ---------------------------------------------
#### ----------------- RUN MSIS2.0 --------------- 
#### ---------------------------------------------

##### Use copy.deepcopy to copy all levels of dictionary and 
###       allow modification of new variable
run_params1 = copy.deepcopy(run_params)
run_params1['den_model'] =  'msis2'  



## Modify the clean_iisset_file() method

In [4]:
import pandas as pd

#### Computer/Command Line functions
import os
import os.path
import sys
import subprocess
import shutil
import linecache
import time


class edit_ICESat2_setup(Pygeodyn):
#     def __init__(self):
#         pass
    
    
    def clean_iisset_file(self):
        '''
        Overwrite the setup file with the icesat2 specific run parameters.

        To make major changes to this function (i.e. implemement a NON-PCE based run of ICESat2)
            construct a new class to inherit this one, and overwrite this method in that class. 
        
        This function does the following:
            - copies setup file to a temporoary file
            - Adds the GLOBAL TITLE Cards (3 strings at top)
            - Removes all instances of the GPS satellites
            - Deletes specified cards in the cards_to_remove list
            - Modifies the cards in card_strings dict
            - Includes the time dependent DRAG options in the card_drag_strings dict
            - Adds cards that are wanted that are not in the file.

            * this

        '''
#         print('REDIFINE WORKED!')
        self.verboseprint('ICESat2 -- clean_iisset_file()')

        #### --------------------------------------------------------------------
        #### Initialize our variables from user input
        (path_to_setupfiles, setup_file_arc, SAT_ID, den_model_setupval) = ( self.INPUTDIR,  self.setup_file_arc, self.SATID, self.iisset_den)      
        
        ORIG_iisset_file = self._INPUT_filename 
        iisset_file      = 'cleaned_setup'+'_'  + self.arcdate_for_files

        #### --------------------------------------------------------------------
        ##### COPY THE FILE SO THAT YOU DON'T OVERWRITE THE ORIGINAL
        ####    We copy to a temporary file "cleaned_setup_file"
        
            
        shutil.copyfile(ORIG_iisset_file, self.TMPDIR_arc +'/'+iisset_file+'.bz2')
        
        os.chdir(self.TMPDIR_arc)
        os.system('bunzip2 -v '+ '*.bz2')
        os.chdir('/data/geodyn_proj/pygeodyn')
        
        iisset_file = self.TMPDIR_arc+'/' +'cleaned_setup'+'_'  + self.arcdate_for_files
#         print(iisset_file)
        
        #### --------------------------------------------------------------------
        #### identify the cards we do not want in the setup file
        cards_to_remove = [ 'ACCEL9',
                            'XEPHEM',
                            'REFRAC',
                            'GPSMOD',
                            'OFFSET',
                            'OFFADJ',
                            'ANTPHC',
                            'ANTPH2',
                            'CGMASS',
                            'OLOAD',
                            'DRAG             5041144 ',       # remove the drag effects on the GPS satellites  
                            'DRAG             5044284',
                            'DRAG             5051204',
                            'DRAG             5154184',
                            'DRAG             5345214',
                            'DRAG             5347224',
                            'DRAG             5356164',
                            'DRAG             5459194',
                            'DRAG             5460234',
                            'DRAG             5461024',
                            'DRAG             5553175',
                            'DRAG             5652315',
                            'DRAG             5658125',
                            'DRAG             5755155',
                            'DRAG             5757295',
                            'DRAG             5848075',
                            'DRAG             5950055',
                            'DRAG             6062256',
                            'DRAG             6163016',
                            'DRAG             6265246',
                            'DRAG             6366276',
                            'DRAG             6464306',
                            'DRAG             6467066',
                            'DRAG             6468096',
                            'DRAG             6469036',
                            'DRAG             6571266',
                            'DRAG             6572086',
                            'DRAG             6573106',
                            'DRAG             6649045',
                            'DRAG             6670326',
                            'DRAG             9743134',
                            'DRAG             9946114',
                            'DRAG   0 0',
                            'MBIAS',
                           # 
                            'SATPAR',
                            'EPOCH',
                            'ELEMS1',
                            'ELEMS2',
                           #
                            'ORBTVU',
                            'RESID',
                          ] 
        #### --------------------------------------------------------------------
        ##### Grab the EPOCH start and end times
        EPOCH_lines = []
        with open(iisset_file, 'r') as f:
            for line_no, line_text in enumerate(f):
                if 'EPOCH         ' in line_text:
                    EPOCH_lines.append(line_no) 

        #### --------------------------------------------------------------------
        ##### Identify and save the EPOCH start and end times
        for i,val in enumerate(EPOCH_lines):
            satpar_line = linecache.getline(iisset_file,val) # Check the above SATPAR line get the correct satellite ID (i.e. NOT GPS)

            ##### only do this for the main satellite, so look for the correct SATID in the SATPAR card above EPOCH
            if SAT_ID in satpar_line:
                epoch_start = linecache.getline(iisset_file,val + 1)[20:40].strip() #181013210000.0000000
                epoch_start_YYMMDD = linecache.getline(iisset_file,val + 1)[20:26].strip()       # 181013
                epoch_start_HHMM = linecache.getline(iisset_file,val + 1)[26:30].strip()         # 2100
                epoch_start_SS_SSSSSSS = linecache.getline(iisset_file,val + 1)[30:40].strip()   # 00.0000000     

                epoch_end   = linecache.getline(iisset_file,val + 1)[60:80].strip() #1810160300 00.000
                epoch_end_YYMMDD = linecache.getline(iisset_file,val + 1)[60:66].strip()       # 181016
                epoch_end_HHMM = linecache.getline(iisset_file,val + 1)[66:70].strip()         # 210000.0000000
                epoch_end_SS_SSSSSSS = linecache.getline(iisset_file,val + 1)[70:80].strip()   # 00.0000000     


        #### --------------------------------------------------------------------
        #### Use pandas datetime and time delta to make adjustments to the dates on the ATGRAV and DRAG cards
        #### --------------------------------------------------------------------
        epoch_start_dt = pd.to_datetime( epoch_start_YYMMDD+epoch_start_HHMM, format='%y%m%d%H%M%S')
        epoch_end_dt = pd.to_datetime( epoch_end_YYMMDD+epoch_end_HHMM, format='%y%m%d%H%M%S')

        ##### ICESat2 has an orbit period of 94.22 minutes
        #### Lets adjust the drag coefficient every 96 minutes
        
        
#         drag_date_rm = (epoch_start_dt+add_mins_dt*15).dt.strftime('%y%m%d%H%M%S').values[0]

        dt_2days = pd.Series(pd.to_timedelta(48,'h'))
        dt_1days = pd.Series(pd.to_timedelta(24,'h'))
        
        dt_epoch_start_minus2days = (epoch_start_dt - dt_2days).dt.strftime('%y%m%d%H%M%S.0000000').values[0]
        dt_epoch_end_plus1days    = (epoch_end_dt + dt_1days).dt.strftime('%y%m%d%H%M%S.000').values[0]
        
        ##### -------------------------------------------------------------------------------------------
        ##### -------------------------------------------------------------------------------------------
        ##### -------------------------------------------------------------------------------------------
        ###       FIND THE X,Y,Z,Xdot,Ydot,Zdot for this epoch start in the PCE data.
        ##### -------------------------------------------------------------------------------------------
#         os.system('bunzip2'+' '+self.StateVector_epochs_datafile+'.bz2')
        
        epoch_start_dt_STR = str(epoch_start_dt)
        date_in_file_flag = False
        
        print("Epoch Start: ", epoch_start_dt_STR)

        with open(self.StateVector_epochs_datafile, 'r') as f:
            for line_no, line_text in enumerate(f):
                
                if epoch_start_dt_STR in line_text:
                    date_in_file_flag= True
#                     print('    ','xyzline',line_no,line_text)

                    break
           
        if date_in_file_flag == False:
            change_elems_flag = False
            print(epoch_start_dt_STR,'not found in file.  Leaving ELEMS as is.')
#             print('Check that the start date:',epoch_start_dt_STR)
#             print('    is within the PCE date range saved in the file')
#             print('       ',self.StateVector_epochs_datafile)
#                     os.system('bzip2'+' '+'/data/data_geodyn/inputs/icesat2/setups/StateVector_epochs.txt')
#             sys.exit()

        else:
            change_elems_flag = True
            xyzline = pd.read_csv(self.StateVector_epochs_datafile, 
                        skiprows = line_no, 
                        nrows=1,           
                        sep = '\s+',
                        dtype=str,
                        names = [
                            'Date',
                            'MJDSECs', 
                            'RSECS', #(fractional secs)
                            'GPS offset', # (UTC - GPS offset (secs))
                            'X',
                            'Y',
                            'Z',
                            'X_dot',
                            'Y_dot',
                            'Z_dot',
                            'YYMMDDhhmmss',
                                ],)

            X     =  xyzline['X'].values[0].ljust(20)     #'  -745933.8926940708'
            Y     =  xyzline['Y'].values[0].ljust(20)     #'  -4864983.834066438'
            Z     =  xyzline['Z'].values[0].ljust(20)     #'    4769954.60524261'
            X_dot =  xyzline['X_dot'].values[0].ljust(20) #'  457.44564954037634'
            Y_dot =  xyzline['Y_dot'].values[0].ljust(20) #'   5302.381564886811'
            Z_dot =  xyzline['Z_dot'].values[0].ljust(20) #'    5463.55571622269'

    #         os.system('bzip2'+' '+self.StateVector_epochs_datafile)
            ##### -------------------------------------------------------------------------------------------
            #### --------------------------------------------------------------------------------------------

        
        ####   INPUT THE OPTIONS ON THE SPECIFIC CARDS YOU WANT TO CHANGE
        ##### Putting in the options is one of the hardest parts of using GEODYN
        #####    They require VERY specific inputs depending on the run type.  
        card_strings = {}
        
        
            #####  ORBFIL KEY ------ Requests output of trajectory file(s) on specified unit(s) 
            #####                           on the last iteration of the run.
            #####
            #####   columns      Orbit output option
            #####    7           Coordinate system of output
            #####                      0 - True of date (default)
            #####                      1 - True of reference date 
            #####                   ** 2 - Mean of year 2000    
            #####    8           Switch indicating whether trajectory file is for a single 
            #####                  satellite or a set of satellites.
            #####                   ** 0 - Single satellite 0 0
            #####                      1 - Set of satellites. This option has meaning 
            #####                            only when used in conjunction with sets of 
            #####                            satellites (See EPOCH and SLAVE option cards
            #####                            for more details ). If satellite ID in columns
            #####                            18-24 is a master satellite , then the trajectory
            #####                          for all satellites in the set will be output.
            #####  9-11           Mandatory unit number for trajectory file. All trajectory 
            #####                  files within an arc must have unique unit numbers. 
            #####                  The suggested unit number starts at 130.
            #####  18-25        Satellite ID. This field must contain a valid ID.
            #####  25-44        START date and time for trajectory output (YYMMDDHHMMSS.SS).
            #####  45-59        STOP  date and time for trajectory output (YYMMDDHHMMSS.SS).
            #####  60-72        Time interval between successive trajectory outputs.

            
#                                  12345678901234567 
        card_strings['ORBFIL'] =  'ORBFIL20131      '+SAT_ID+'     '+str(epoch_start)[:-6]+'  '+str(epoch_end)[:6]+' 24200.00          60'
        card_strings['RESID']  =  'RESIDU12'
        card_strings['OBSVU']  =  'OBSVU 2'  # print residuals on last iteration only
        #       card_strings['PRNTVU'] =  'PRNTVU55212222    22122'  # original
        card_strings['PRNTVU'] =  'PRNTVU5521111211 121122'  # suppress some IIS/IIE outputs.
#                                  1234567890 
        card_strings['ORBTVU'] =  'ORBTVU1201       '+SAT_ID+'     '+str(epoch_start)[:-6]+'  '+str(epoch_end)[:6]+' 24200.00 .100000D+01'
        card_strings['ATMDEN'] =  'ATMDEN  '+ den_model_setupval
        card_strings['ATGRAV']  =  'ATGRAV9090              '+dt_epoch_start_minus2days +''+dt_epoch_end_plus1days[:-1]   
        card_strings['I64G2E']  =  'I64G2E         25'  # using 30 like in st-SLR run maxed out the memory usage
        card_strings['SIGMA           1']  =  'SIGMA           1               1.0                 1.0'    
        card_strings['SIGMA           2']  =  'SIGMA           2               1.0                 1.0'    
        card_strings['SIGMA           3']  =  'SIGMA           3               1.0                 1.0'   
        card_strings['SIGMA          51']  =  'SIGMA          51               10.0D+25             0.10'  
        card_strings['SIGMA          85']  =  'SIGMA          85               0.010000            0.010000'  
        

        ### Fix the coordinate system... PCE Data was in J2000
#         card_strings['REFSYS1933 0        ']  = 'REFSYS193310        '+epoch_start+'0'
#         card_strings['SATPAR   13']  =  'SATPAR   139     '+SAT_ID+'          9.53000000       1514.000'
        card_strings['REFSYS']  = 'REFSYS193310        '+epoch_start+'0'
        card_strings['EPOCH'] = 'EPOCH               '+epoch_start+epoch_start+epoch_end
        card_strings['SATPAR']  =  'SATPAR   139     '+SAT_ID+'          9.53000000       1514.000'
        
        if change_elems_flag == True:
            card_strings['ELEMS1']  = 'ELEMS11             '+X+''+Y+''+Z+''   
            card_strings['ELEMS2']  = 'ELEMS2              '+X_dot+''+Y_dot+''+Z_dot+''
                
        
        #### Suppress the printing of the flux model
        card_strings['FLUX  1']  =  'FLUX  0'

    

        #### --------------------------------------------------------------------
        ####    Search through the file to see if any of the cards we WANT are NOT in the file
        #### --------------------------------------------------------------------
        ##### read in all lines of the file and save them
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()                
        ##### card flags to see if certain cards are present in the file
        card_flag = {}
        for card in card_strings:
            ### Set the default flag to be False,  if the card is in the file, flip the flag to True
            card_flag[card] = False
            for line in lines_all:
                if card in line:
                    card_flag[card] = True

        #### --------------------------------------------------------------------
        ####    Edit the cards that exist in the file that we want to modify
        #### --------------------------------------------------------------------
        ###### Re-write the file line-by-line and EDIT the cards that need to be modified    
        lines_replace = {}
        with open(iisset_file, "r") as f:
            lines = f.readlines()
            for line_num, line in enumerate(lines):
                for card in card_strings:
                    if card in line:
                        lines_replace[line_num] = card_strings[card]
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()
        with open(iisset_file, "w") as f:
            for line_num, line in enumerate(lines_all):
                if line_num in lines_replace:
#                     print('replacing line',lines_replace[line_num])
                    f.write(lines_replace[line_num]+'\n')
                else:
                     f.write(line)


        #### for adding time dependent drag estimations.  We need to do a few things:
        ###       Find the drag card that is already in the file:
        ###       Add CONDRAG before all drag cards
        ###       Add DRAG cards with TIME periods after the first drag card
        
        
                #### --------------------------------------------------------------------
        ####   INPUT THE DRAG OPTIONS  for time dependent drag
        
        #         add_hours_dt = pd.Series(pd.to_timedelta(9,'h'))
#         add_mins_dt = pd.Series(pd.to_timedelta(94.22,'m'))
        add_mins_dt = pd.Series(pd.to_timedelta(4,'h'))
    
        drag_date_1 =  (epoch_start_dt+ add_mins_dt*1).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_2 =  (epoch_start_dt+ add_mins_dt*2).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_3 =  (epoch_start_dt+ add_mins_dt*3).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_4 =  (epoch_start_dt+ add_mins_dt*4).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_5 =  (epoch_start_dt+ add_mins_dt*5).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_6 =  (epoch_start_dt+ add_mins_dt*6).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_7 =  (epoch_start_dt+ add_mins_dt*7).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_8 =  (epoch_start_dt+ add_mins_dt*8).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_9 =  (epoch_start_dt+ add_mins_dt*9).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_10 = (epoch_start_dt+add_mins_dt*10).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_11 = (epoch_start_dt+add_mins_dt*11).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_12 = (epoch_start_dt+add_mins_dt*12).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_13 = (epoch_start_dt+add_mins_dt*13).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_14 = (epoch_start_dt+add_mins_dt*14).dt.strftime('%y%m%d%H%M%S').values[0]
        drag_date_15 = (epoch_start_dt+add_mins_dt*15).dt.strftime('%y%m%d%H%M%S').values[0]
        
        
        
        print('epoch end   :',epoch_end[:-5])
        print('epoch start :',epoch_start[:-5])
                
        card_drag_strings={}                                                                                                           #11307.
        card_drag_strings['CONDRG']  =  'CONDRG  1        '+SAT_ID+'     '+str(epoch_start[:-5])+str(epoch_end[:-5])+'         0.80000  14400.'
#         card_drag_strings['DRAG   0 0       '+SAT_ID+' 2.3000000000000E+00']  =  'DRAG   0 0       '+SAT_ID+' 2.3000000000000E+00'
        card_drag_strings[drag_date_1]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_1[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_2]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_2[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_3]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_3[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_4]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_4[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_5]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_5[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_6]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_6[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_7]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_7[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_8]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_8[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_9]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_9[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_10]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_10[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_11]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_11[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_12]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_12[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_13]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_13[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_14]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_14[:10]+' 0.00    0.100D+02'
        card_drag_strings[drag_date_15]  = 'DRAG             '+SAT_ID+' 2.2000000000000D+00'+drag_date_15[:10]+' 0.00    0.100D+02'
        
        
        print(drag_date_1)
        print(drag_date_2)
        print(drag_date_3)
        print(drag_date_4)
        print(drag_date_5)
        print(drag_date_6)
        print(drag_date_7)
        print(drag_date_8)
        print(drag_date_9)
        print(drag_date_10)
        print(drag_date_11)
        print(drag_date_12)
#         print(drag_date_13)
#         print(drag_date_14)
#         print(drag_date_15)
        
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()                
        with open(iisset_file, "w") as f:
            for line in lines_all:
                if 'DRAG   0 0       '+SAT_ID+' 2.3000000000000E+00' in line:  #this finds the DRAG line.  
                    f.write(card_drag_strings['CONDRG'] + ' \n')
                    f.write('DRAG             '+SAT_ID+' 2.2000000000000E+00'+ ' \n')
                    f.write(card_drag_strings[drag_date_1] + ' \n')                 
                    f.write(card_drag_strings[drag_date_2] + ' \n')                 
                    f.write(card_drag_strings[drag_date_3] + ' \n')                 
                    f.write(card_drag_strings[drag_date_4] + ' \n')                 
                    f.write(card_drag_strings[drag_date_5] + ' \n')                 
                    f.write(card_drag_strings[drag_date_6] + ' \n') 
                    f.write(card_drag_strings[drag_date_7] + ' \n')                 
                    f.write(card_drag_strings[drag_date_8] + ' \n')                 
                    f.write(card_drag_strings[drag_date_9] + ' \n')                 
                    f.write(card_drag_strings[drag_date_10] + ' \n')                 
                    f.write(card_drag_strings[drag_date_11] + ' \n')                 
                    f.write(card_drag_strings[drag_date_12] + ' \n')                 
#                     f.write(card_drag_strings[drag_date_13] + ' \n')                 
#                     f.write(card_drag_strings[drag_date_14] + ' \n')                 
#                     f.write(card_drag_strings[drag_date_15] + ' \n')                 

                else:
                    f.write(line)
                    
              
        #####-----------------------------------------------------------------------------
        #####   Delete the SATPAR GPS, EPOCH, ELEMS110, and ELEMS2 lines after the SATPAR GPS
        #####   Do this by finding the SATPAR for our sat and then saving it and the next 3 lines
        #####   then delete all the SATPAR,EPOCH,ELEMS110, ELEMS2 and restore the ones we saved
        #####-----------------------------------------------------------------------------

        if change_elems_flag == False:
            ##### read in all lines of the file and save them
            with open(iisset_file, "r") as f:
                lines_all = f.readlines()    
            
            for iline, line in enumerate(lines_all):
                if 'ELEMS1'in line:
                    save_ELEMS1 = iline+1
                elif 'ELEMS2' in line:
                    save_ELEMS2 = iline+1
                
            line_ELEMS1 = linecache.getline(iisset_file, save_ELEMS1)
            line_ELEMS2  = linecache.getline(iisset_file, save_ELEMS2)
            
            card_strings['ELEMS1']  = line_ELEMS1
            card_strings['ELEMS2']  = line_ELEMS2
            
#             print(line_ELEMS1)
#             print(line_ELEMS2)

        ####----------------------------------------------------------------------
        #### REMOVE CARDS:: rewrite the file without the cards we specified in the cards_to_remove dict
        ####----------------------------------------------------------------------

        ##### read in all lines of the file and save them
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()    
        ##### Re-write the file line-by-line WITHOUT the cards that we want to remove    
        with open(iisset_file, "w") as f:
            for iline, line in enumerate(lines_all):
                if any(card in line for card in cards_to_remove):
                    # IF the any of the cards in the list are in the line, dont add it
                    pass
                else:
                    f.write(line)                
        
                        
        
        ####----------------------------------------------------
        #### Add any cards that we want that are not in the file
        ##### this INCLUDES our saved 
        #####      SATPAR, EPOCH, ELEMS1, ELEMS2
        #####---------------------------------------------------------
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()                  

        # use some switches to determine if things have already been written in the loop and avoid writing too many
        switch_cardcount = 0
        switch_2     = True
        
        for card in card_flag:
            if card_flag[card] == False:
                with open(iisset_file, "w") as f:
                    for line in lines_all:
                        if 'ALBEDO' in line:  #this overwrites the line after albedo. 
                            # MAYBE TODO:  this may not write multiple cards that aren't in the file
                            if switch_cardcount == 0:
                                f.write(line)
                                f.write(card_strings[card] + ' \n') 
                            else: 
                                f.write(card_strings[card] + ' \n')
                                switch_cardcount += 1
                        else:
                            f.write(line)
                            
        ##### Write our satellite parameters back in         
        with open(iisset_file, "w") as f:
            for line in lines_all:
                if (('REFSYS' in line) and (switch_2 == True)):
                    f.write(card_strings['REFSYS']  + ' \n')
                    f.write(card_strings['SATPAR']  + ' \n')
                    f.write(card_strings['EPOCH']   + ' \n')
                    if change_elems_flag == True:
                        f.write(card_strings['ELEMS1']  + ' \n')
                        f.write(card_strings['ELEMS2']  + ' \n')
                    elif change_elems_flag == False:
                        f.write(card_strings['ELEMS1'])
                        f.write(card_strings['ELEMS2'])

                    switch_2 = False
                else:
                    f.write(line)

                    
#         self.verboseprint('    ','Orig:')
#         self.verboseprint('    ','    ',line_ELEMS11.rstrip('\n'))
#         self.verboseprint('    ','    ',line_ELEMS2.rstrip('\n'))
        self.verboseprint('    ','PCE Update:')
        self.verboseprint('    ','    ',card_strings['ELEMS1'])
        self.verboseprint('    ','    ',card_strings['ELEMS2'])
        
        ####----------------------------------------------------------------------
        #### Add three lines to the start of the file.  This is the GLOBAL TITLE
        ####----------------------------------------------------------------------

        with open(iisset_file, 'r+') as f:
            content = f.read()
            f.seek(0, 0)   # find the first lines
            f.write('### \n') 
            f.write('###   '+self.arc_name_id+'  \n') 
            f.write('### \n') 
            f.write(content) 
            
            
        ####----------------------------------------------------------------------
        #### Try doing a complete job of removing the GPS satellites.
        ####----------------------------------------------------------------------
            
        delete_gps_sats = [ '5041144',
                            '5044284',
                            '5051204',
                            '5154184',
                            '5345214',
                            '5347224',
                            '5356164',
                            '5459194',
                            '5460234',
                            '5461024',
                            '5553175',
                            '5652315',
                            '5658125', 
                            '5755155',
                            '5757295',
                            '5848075',
                            '5950055',
                            '6062256',
                            '6163016',
                            '6265246',
                            '6366276',
                            '6464306',
                            '6467066',
                            '6468096',
                            '6469036',
                            '6571266',            
                            '6572086',
                            '6573106',
                            '6649045',
                            '6670326',
                            '9743134',
                            '9946114',        
                            ]
        ##### read in all lines of the file and save them
        with open(iisset_file, "r") as f:
            lines_all = f.readlines()    
        ##### Re-write the file line-by-line WITHOUT the cards that we want to remove    
        with open(iisset_file, "w") as f:
            for iline, line in enumerate(lines_all):
                if any(gps in line for gps in delete_gps_sats):
                    # IF the any of GPS IDs in the list are in the line, dont add it the line
                    pass
                else:
                    f.write(line)      



## Run

In [5]:
## Construct object with run params
# Obj_Geodyn = edit_ICESat2_setup(run_params1)

## Run pyeodyn for the arcs in the above set.
# Obj_Geodyn.RUN_GEODYN()



## Read the Data back in

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYN import Pygeodyn


read_params = copy.deepcopy(run_params1)
read_params['action']  =  'read'  
read_params['den_model']  =  'msis2'  
read_params['accels']  =  True
read_params['request_data']      = ['AdjustedParams',
                                    'Trajectory_orbfil',
                                    'Density',
                                    'Residuals_obs',
#                                     'Residuals_summary',
                                   ]
### Load the data into an object
Obj_Geodyn = 0  # clear out the object variable if its got stuff in it.
Obj_Geodyn = Pygeodyn(read_params)
Obj_Geodyn.getData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                      ......... READING GEODYN output
     Loading ... icesat2_2018319_54hr.msis2 


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-d64c6f6f9423>", line 20, in <module>
    Obj_Geodyn.getData()
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_Read.py", line 2006, in getData
    self.check_if_run_converged(self._iieout_filename)
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/util_classtools.py", line 680, in check_if_run_converged
    if 'CONVERGENCE' in line:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above ex

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

--- Logging error ---


Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-d64c6f6f9423>", line 20, in <module>
    Obj_Geodyn.getData()
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_Read.py", line 2006, in getData
    self.check_if_run_converged(self._iieout_filename)
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/util_classtools.py", line 680, in check_if_run_converged
    if 'CONVERGENCE' in line:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above ex

Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-d64c6f6f9423>", line 20, in <module>
    Obj_Geodyn.getData()
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/PYGEODYN_Read.py", line 2006, in getData
    self.check_if_run_converged(self._iieout_filename)
  File "/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/util_classtools.py", line 680, in check_if_run_converged
    if 'CONVERGENCE' in line:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above ex

In [5]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
#                 'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                })





from datetime import datetime,timedelta

# Simplify Plotting Schemes:
col1 = px.colors.qualitative.Plotly[0]
col2 = px.colors.qualitative.Plotly[1]
col3 = px.colors.qualitative.Plotly[2]

def Calc_Cd_percent_diff_apriori(obj_m1):
    '''
    CALCULATE:  
    '''

    SAT_ID = int(obj_m1.__dict__['global_params']['SATID'])
    which_stat = 'CURRENT_VALUE'
    model_m1 = obj_m1.__dict__['global_params']['den_model']

    all_cd_m1    = []
    all_dates_m1 = []

    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        i_arc = ii+1
#         print(obj_m1.AdjustedParams[arc])
        last_iter = list(obj_m1.AdjustedParams[arc].keys())[-1]
        labels = list(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'].keys())

        for i,val in enumerate(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'].keys()):
            all_cd_m1.append(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][val][which_stat])
            all_dates_m1.append(labels[i])

    # take % difference from a priori
    cd_apriori= 2.2
    percdiff_cd_m1 = ((np.array(all_cd_m1) - cd_apriori)/ cd_apriori)*100

    obj_m1_stats = {}
    obj_m1_stats['cd_apriori'] = cd_apriori
    obj_m1_stats['cd_percdiff_from_apriori'] = percdiff_cd_m1
    obj_m1_stats['all_cd'] = all_cd_m1
    obj_m1_stats['all_dates'] = all_dates_m1

    return(obj_m1_stats)


ERROR! Session/line number was not unique in database. History logging moved to new session 2325


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1d1adf32e98b>", line 4, in <module>
    import plotly.express as px
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/express/__init__.py", line 16, in <module>
    from ._chart_types import (  # noqa: F401
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/express/_chart_types.py", line 800, in <module>
    line_ternary.__doc__ = make_docstring(line_ternary)
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/express/_doc.py", line 559, in make_docstring
    tw.fill(" ".join(param_desc_list or ""))
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/textwrap.py", line 363, in fill
    return "\n".join(self.wrap(text))
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/textwrap.py


KeyboardInterrupt



In [ ]:
obj_m1  = Obj_Geodyn
plot_num = 0


# def plot_cd_and_cdratio(fig, obj_m1, plot_num):
obj_m1_stats = Calc_Cd_percent_diff_apriori(obj_m1)

if plot_num == 0:
    col = col1
    x_annot = 1.05
    y_annot = .90
    m_size = 2

elif plot_num == 1:
    x_annot = 1.05
    y_annot = .8
    col = col2
    m_size = 2

elif plot_num == 2:
    x_annot = 1.05
    y_annot = .65 
    col = col3    
    m_size = 2

SAT_ID = int(obj_m1.__dict__['global_params']['SATID'])
which_stat = 'CURRENT_VALUE'


model_m1 = obj_m1.__dict__['global_params']['den_model']
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(["Timeseries of Cd", "A priori-to-adjusted Cd Ratio"]),
    vertical_spacing = 0.15,
    )

for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
#         print(arc)

    str_run_param = 'run_parameters'+ arc
    final_iter = obj_m1.__dict__[str_run_param]['str_iteration']

    i_arc = ii+1
    last_iter = list(obj_m1.AdjustedParams[arc].keys())[-1]
    time_dep_cd_dates = list(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'].keys())
    
    val_list_1 = []


    for i in time_dep_cd_dates:
#         print(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat])
        val_list_1.append(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat])

        cd_ratio =  obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat]/ obj_m1_stats['cd_apriori'] 
        fig.add_trace(go.Scattergl(x=  [pd.to_datetime(i)],
                                   y=  [cd_ratio],
                           name= model_m1,
                           mode='markers',
                           marker=dict(
                           color=col,
                           size=7,
                           ),
                           showlegend=False,
                           ),
                           row=2, col=1,)

    #### FIRST PLOT (CD TIMESERIES and APRIORI CD)
    fig.add_trace(go.Scattergl(x=time_dep_cd_dates,
                               y=val_list_1,
                               name= model_m1 ,#+ ' |  Arc ' +str(i_arc) +' | Iters: '+ str(last_iter),
                               mode='markers',
                               marker=dict(
                               color=col,
                               size=7,),
                               showlegend=False,
                               ),
                               row=1, col=1,
                               )
    fig.add_trace(go.Scattergl(x=time_dep_cd_dates,
                               y=obj_m1_stats['cd_apriori']*np.ones(np.size(time_dep_cd_dates)),
                               name= 'A priori Cd',
                               mode='lines+markers',
                               marker=dict(
                               color='black',
                               size=5,),
                               showlegend=False,
                               ),
                               row=1, col=1,
                               )
    
    


    arc_date_1 = obj_m1.__dict__['global_params']['arc_input'][ii]

    arc_date_1 = datetime.strptime(arc_date_1, '%Y.%j')
    arc_date_2 = arc_date_1 + (pd.to_timedelta(24,'h'))
#         fig = add_arc_background_w_text(fig, 2.2, arc_date_1, arc_date_2, i_arc, False)

#### SECOND PLOT (PERC diff b/w apriori and Cd)
#     fig = legend_as_annotation(fig, obj_m1.__dict__['global_params']['den_model'], col, x_annot, y_annot)


# fix layout info:
fig.update_yaxes( title="Cd ",  exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="ratio",exponentformat= 'power',row=2, col=1)
fig.update_xaxes( title="Date", row=2, col=1)

fig.update_layout(title="Time Dependent Drag Coefficient ")
fig.update_layout(
                autosize=False,
                width=900,
                height=700,
                font=dict(size=14))




In [ ]:
def orb_avg(den_df, arc):
    lat = np.asarray(den_df[arc]['Lat'])
    time_pd = pd.to_datetime(den_df[arc]['Date'])
    i = np.nonzero( lat[1:]*lat[0:-1]  <  np.logical_and(0 , lat[1:] > lat[0:-1] )  )
    i = i[0]

    d_avg = np.zeros(np.size(i))
    height_avg = np.zeros(np.size(i))
    time_avg = []

    d_avg_rolling = []
    
    roll_avg_count = 0
    for j in range(np.size(i)-1):
    #     print(j+1)
        d_avg[j] = np.mean(  den_df[arc]['rho (kg/m**3)'][   i[j] : i[j+1]-1  ]  )
        height_avg[j] = np.mean(  den_df[arc]['Height (meters)'][   i[j] : i[j+1]-1  ]  )
        mean_time=time_pd[   i[j] : i[j+1]-1  ].mean() 
        time_avg.append(  mean_time)

        if roll_avg_count ==2:
            d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
            
            roll_avg_count =0
            
        roll_avg_count+=1 
        
    return(time_avg, d_avg, d_avg_rolling )


In [ ]:
# obj_m1  = Obj_Geodyn
# plot_num = 0


# # def plot_cd_and_cdratio(fig, obj_m1, plot_num):
# obj_m1_stats = Calc_Cd_percent_diff_apriori(obj_m1)

# if plot_num == 0:
#     col = col1
#     x_annot = 1.05
#     y_annot = .90
#     m_size = 2

# elif plot_num == 1:
#     x_annot = 1.05
#     y_annot = .8
#     col = col2
#     m_size = 2

# elif plot_num == 2:
#     x_annot = 1.05
#     y_annot = .65 
#     col = col3    
#     m_size = 2

# SAT_ID = int(obj_m1.__dict__['global_params']['SATID'])
# which_stat = 'CURRENT_VALUE'


# model_m1 = obj_m1.__dict__['global_params']['den_model']
# fig = make_subplots(
#     rows=3, cols=1,
#     subplot_titles=(["Density", 'F10.7 Flux', 'Kp']),
#     vertical_spacing = 0.15,
#     )

# for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
# #         print(arc)
#     time_avg, d_avg, d_avg_rolling = orb_avg(obj_m1.__dict__['Density'], arc)
#     fig.add_trace(go.Scattergl(x=obj_m1.__dict__['Density'][arc]['Date'],
#                                y=obj_m1.__dict__['Density'][arc]['rho (kg/m**3)'],
#                                name= 'Density',
#                                mode='markers',
#                                marker=dict(
#                                color=col,
#                                size=3,),
#                                showlegend=False,
#                                ),
#                                row=1, col=1,
#                                )
#     fig.add_trace(go.Scattergl(x=obj_m1.__dict__['Density'][arc]['Date'],
#                                y=obj_m1.__dict__['Density'][arc]['flux_daily'],
#                                name= 'Daily',
#                                mode='markers',
#                                marker=dict(
#                                color='grey',
#                                size=4,),
#                                showlegend=False,
#                                ),
#                                row=2, col=1,
#                                )
#     fig.add_trace(go.Scattergl(x=obj_m1.__dict__['Density'][arc]['Date'],
#                                y=obj_m1.__dict__['Density'][arc]['flux_avg'],
#                                name= '81 day avg',
#                                mode='markers',
#                                marker=dict(
#                                color='black',
#                                size=4,),
#                                showlegend=False,
#                                ),
#                                row=2, col=1,
#                                )
#     fig.add_trace(go.Scattergl(x=obj_m1.__dict__['Density'][arc]['Date'],
#                                y=obj_m1.__dict__['Density'][arc]['Kp'],
#                                name= '81 day avg',
#                                mode='markers',
#                                marker=dict(
#                                color='black',
#                                size=4,),
#                                showlegend=False,
#                                ),
#                                row=3, col=1,
#                                )
# #     fig.add_trace(go.Scattergl(x=time_avg,
# #                                y=d_avg,
# #                                name= 'A priori Cd',
# #                                mode='markers',
# #                                marker=dict(
# #                                color='red',
# #                                size=8,),
# #                                showlegend=False,
# #                                ),
# #                                row=1, col=1,
# #                                )


#     str_run_param = 'run_parameters'+ arc
#     final_iter = obj_m1.__dict__[str_run_param]['str_iteration']

    
# # fix layout info:
# fig.update_yaxes(type="log", title="Cd ",  exponentformat= 'power',row=1, col=1)
# fig.update_yaxes( title="ratio",exponentformat= 'power',row=2, col=1)
# fig.update_xaxes( title="Date", row=2, col=1)

# fig.update_layout(title="Time Dependent Drag Coefficient ")
# fig.update_layout(
#                 autosize=False,
#                 width=900,
#                 height=600,
#                 font=dict(size=14))




In [ ]:
# val_list_1

In [ ]:
# time_dep_cd_dates = list(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'].keys())
# for i in time_dep_cd_dates:
#     print(obj_m1.AdjustedParams['2018.318'][13][SAT_ID]['0CD'][i]['CURRENT_VALUE'])


In [ ]:
# %load_ext autoreload
# %autoreload 2
# from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_cd_and_percdiff_from_apriori


# fig = make_subplots(
#     rows=2, cols=1,
#     subplot_titles=(["Timeseries of Cd", "Cd Ratio (a priori/adjusted value)"]),
#     vertical_spacing = 0.08,
#     )
# fig = plot_cd_and_cdratio(fig,  Obj_Geodyn, 0)
# # fig = plot_cd_and_cdratio(fig,  Obj_Geodyn3, 1)
# # fig = plot_cd_and_cdratio(fig,  Obj_Geodyn1, 2)


# fig.show(config=config)

In [ ]:
# %load_ext autoreload
# %autoreload 2
# from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_cd_and_percdiff_from_apriori


# fig = make_subplots(
#     rows=2, cols=1,
#     subplot_titles=(["Timeseries of Cd", "Percent difference from a priori (Cd=2.2)"]),
#     vertical_spacing = 0.08,
#     )
# fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn, 0)
# # fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn3, 1)
# # fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn1, 2)


# fig.show(config=config)

In [ ]:
# %load_ext autoreload
# %autoreload 2

# from PYGEODYNAnalysis_icesat2PCEtrajectory import ARCOVERLAP_2arcs_ObsResids_NTW_intrack

# fig = make_subplots(rows=2, cols=1, 
#             subplot_titles=(['In-Track Component', 'Residual (PCE-ORBFIL)']),
#             vertical_spacing = 0.2,
#             specs=[ [{"secondary_y": False }],
#                     [{"secondary_y": False }]],)

# arc1 = '2018.319'
# arc2 = arc1

# fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn, 0, arc1, arc2)
# # fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn3, 1, arc1, arc2)
# # fig = ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, Obj_Geodyn1, 2, arc1, arc2)

# fig.show(config=config)


In [ ]:
# last_nonpredicted_time

In [ ]:

# def ARCOVERLAP_2arcs_ObsResids_NTW_intrack(fig, obj_m1, plot_num, arc1, arc2):

import sys
sys.path.insert(0,'/data/geodyn_proj/pygeodyn/utils_pygeodyn_develop/util_dir/')
from common_functions          import Convert_cartesian_to_RSW, Convert_cartesian_to_NTW



###### GET THE PCE DATA:
StateVector_PCE_datafile = '/data/data_geodyn/inputs/icesat2/setups/PCE_ascii.txt'
#     os.system('bunzip2 -v '+ StateVector_epochs_datafile +'.bz2')


fig = make_subplots(rows=2, cols=1,
            shared_xaxes=True,
            subplot_titles=(['Adjusted Cd ratio to a priori (2.2)', 'In-Track Component Residuals (PCE-ORBFIL)']),
            vertical_spacing = 0.1,
            specs=[ [{"secondary_y": False }],
                    [{"secondary_y": False }]],)

arc1 = '2018.319'
arc2 = arc1


first_arc = arc1
last_arc  = arc2
first_arc_first_time = obj_m1.__dict__['Trajectory_orbfil'][first_arc]['data_record']['Date_UTC'].iloc[0],
last_arc_last_time   = obj_m1.__dict__['Trajectory_orbfil'][last_arc]['data_record']['Date_UTC'].iloc[-2]
first_arc_first_time_str =  str(first_arc_first_time[0])#.replace( "'",' ') 
last_arc_last_time =  str(last_arc_last_time)#.replace( "'",' ') 

# print('first_arc_first_time',first_arc_first_time)
####---------------------------------------------------------
with open(StateVector_PCE_datafile, 'r') as f:
    for line_no, line_text in enumerate(f):

        if first_arc_first_time_str in line_text:
            first_line = line_no
        elif last_arc_last_time in line_text:
            last_line = line_no
            break
PCE_data = pd.read_csv(StateVector_PCE_datafile, 
            skiprows = first_line, 
            nrows=last_line- first_line,           
            sep = '\s+',
            dtype=str,
            names = [
                    'Date',
                    'MJDSECs', 
                    'RSECS', #(fractional secs)
                    'GPS offset', # (UTC - GPS offset (secs))
                    'X',
                    'Y',
                    'Z',
                    'X_dot',
                    'Y_dot',
                    'Z_dot',
                    'YYMMDDhhmmss',
                        ],)
#     os.system('bzip2 -v '+ StateVector_epochs_datafile )




####---------------------------------------------------------

PCE_data['Date_pd'] = pd.to_datetime(PCE_data['Date'])

orbfil_arc1 = obj_m1.__dict__['Trajectory_orbfil'][arc1]['data_record']
orbfil_arc1['Date_pd'] = pd.to_datetime(orbfil_arc1 ['Date_UTC'])

orbfil_arc2 = obj_m1.__dict__['Trajectory_orbfil'][arc2]['data_record']
orbfil_arc2['Date_pd'] = pd.to_datetime(orbfil_arc2 ['Date_UTC'])



C_1 = pd.merge(left=orbfil_arc1, left_on='Date_pd',
     right=PCE_data, right_on='Date_pd')
C_2 = pd.merge(left=orbfil_arc2, left_on='Date_pd',
         right=PCE_data, right_on='Date_pd')



X = C_1['Satellite Inertial X coordinate']
Y = C_1['Satellite Inertial Y coordinate']
Z = C_1['Satellite Inertial Z coordinate']
Xdot = C_1['Satellite Inertial X velocity']
Ydot = C_1['Satellite Inertial Y velocity']
Zdot = C_1['Satellite Inertial Z velocity']
state_vector = np.transpose(np.array([X, Y, Z, Xdot, Ydot, Zdot]))

InTrack_comp_orbfil = [Convert_cartesian_to_NTW(x) for x in state_vector]

X = C_1['X'].astype(float)
Y = C_1['Y'].astype(float)
Z = C_1['Z'].astype(float)
Xdot = C_1['X_dot'].astype(float)
Ydot = C_1['Y_dot'].astype(float)
Zdot = C_1['Z_dot'].astype(float)
state_vector = np.transpose(np.array([X, Y, Z, Xdot, Ydot, Zdot]))

InTrack_comp_PCE = [Convert_cartesian_to_NTW(x) for x in state_vector]

model_m1 = obj_m1.__dict__['global_params']['den_model']

if plot_num == 0:
    col = col1
    x_annot = 1.05
    y_annot = .97
    m_size = 3
elif plot_num == 1:
    x_annot = 1.05
    y_annot = .8
    col = col2
    m_size = 2.5
elif plot_num == 2:
    x_annot = 1.05
    y_annot = .55 
    col = col3    
    m_size = 2


#     for i, arc in enumerate([arc1 , arc2]):
#         i_arc = i+1
data_skip = 7
####--------------------- INTRACK Component  ---------------------
# fig.add_trace(go.Scattergl(x=C_1['Date_pd'][::data_skip],
#                          y=InTrack_comp_orbfil[::data_skip],
#                          name= 'Orbfil',
#                          mode='markers',
#                          marker=dict(color=col,
#                          size=m_size,),
#                          showlegend=False,
#                          ),
#                          secondary_y=False,
#                          row=1, col=1,
#                          )

# fig.add_trace(go.Scattergl(x=C_1['Date_pd'][::data_skip],
#                          y=InTrack_comp_PCE[::data_skip],
#                          name= 'PCE',
#                          mode='markers',
#                          marker=dict(color=col, opacity=0.3,
#                          size=m_size+1,),
#                          showlegend=False,
#                          ),
#                          secondary_y=False,
#                          row=1, col=1,
#                          )        

####--------------------- Residual  ---------------------
for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
#         print(arc)

    str_run_param = 'run_parameters'+ arc
    final_iter = obj_m1.__dict__[str_run_param]['str_iteration']

    i_arc = ii+1
    last_iter = list(obj_m1.AdjustedParams[arc].keys())[-1]
    time_dep_cd_dates = list(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'].keys())
    
    val_list_1 = []


    for i in time_dep_cd_dates:
#         print(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat])
        val_list_1.append(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat])

        cd_ratio =  obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat]/ obj_m1_stats['cd_apriori'] 
        fig.add_trace(go.Scattergl(x=  [pd.to_datetime(i)],
                                   y=  [cd_ratio],
                           name= model_m1,
                           mode='markers+text',
                           marker=dict(
                           color=col,
                           size=7,
                           ),
                           text =  'Cd='+str(round(obj_m1.AdjustedParams[arc][last_iter][SAT_ID]['0CD'][i][which_stat], 2)), #str(cd_ratio),
                           textposition="top center",
                           showlegend=False,
                           ),
                           row=1, col=1,)
#         fig.add_trace(go.Scattergl(x=  [pd.to_datetime(i)],
#                                    y=  [cd_ratio],
#                            name= model_m1,
#                            mode='lines',
#                            line=dict(color=col, width=3),
# #                            line= dict(shape= 'hv'), 
#                            showlegend=False,
#                            connectgaps=True,),
#                            row=1, col=1,)        

add_dt = pd.to_timedelta(180,'m')
overlap_end   = obj_m1.__dict__['Trajectory_orbfil'][arc1]['data_record']['Date_UTC'].iloc[-1]

date = pd.to_datetime(i)

while date < overlap_end:
    date += add_dt
    fig.add_trace(go.Scattergl(x= [date] ,
                               y=  list(np.ones(1)),
                               name= model_m1,
                               mode='markers+text',
                               marker=dict(
                               color=col,
                               size=7,
                               ),
                               text =  '2.2', #str(cd_ratio),
                               textposition="bottom center",
                               showlegend=False,
                               ),
                               row=1, col=1,)



resid = (np.array(InTrack_comp_PCE) - np.array(InTrack_comp_orbfil))*1e2

fig.add_trace(go.Scattergl(x=C_1['Date_pd'][::data_skip],
                           y=resid[::data_skip],
                         name= '(PCE-orbfil)',
                         mode='markers',
                         marker=dict(color=col,
                         size=m_size,),
                         showlegend=False,
                         ),
                         secondary_y=False,
                         row=2, col=1,
                         )

### Start of second arc
overlap_start = obj_m1.__dict__['Residuals_obs'][arc1]['Date'].iloc[-1]
### End of first arc
overlap_end   = obj_m1.__dict__['Trajectory_orbfil'][arc1]['data_record']['Date_UTC'].iloc[-1]
fig.add_vrect(  x0=overlap_start, x1=overlap_end,
                fillcolor='LightSkyBlue', opacity=0.2,
                layer="below", line_width=0)

# fig = legend_as_annotation(fig, obj_m1.__dict__['global_params']['den_model'], col, x_annot, y_annot)

fig.update_layout(title="NTW Coord. System + Predicted Window (light blue window)")
fig.update_layout(
                autosize=False,
                width=800,
                height=600,
                font=dict(size=12),
                legend= {'itemsizing': 'constant'})

fig.update_yaxes( title="Cd Ratio (Adjusted Cd/2.2) ",exponentformat= 'power',row=1, col=1)
fig.update_yaxes( title="Residual (cm)",       exponentformat= 'power',row=2, col=1)
fig.update_xaxes( title="Date", row=2, col=1)
fig.update_yaxes(title_text="Residuals (cm)", row=1, col=1, secondary_y=True, color='SkyBlue')
fig.update_traces(textfont_size=10, textfont_color=col)

#     return(fig)


###### 